# OR Case Study

## This case has a total of 4 configurations:
### i) A: no warehouse, no upgrade
#### This is the current situation
### ii) B: warehouses built, 80% demand criterion met, plants not upgraded
### iii) C: no warehouses built, 80% demand criterion not met, all plants upgraded 
### iv) D: warehouses built, 80% demand criterion met, upgradation of plants is also a decision variable - the upgradation decision is included in the objective function (the decision is not included explicitly in the above 3 confs.)



## Getting results for conf. A

### i) comment the constraint for 80% demand in scenario 1 specific constraints
### ii) uncomment the initialization of yp variable when it is created

## Getting results for conf. B

### i) uncomment the constraint for 80% demand in scenario 1 specific constraints
### ii) uncomment the initialization of yp variable when it is created

## Getting results for conf. C

### i) comment the constraint for 80% demand in scenario 1 specific constraints
### ii) comment the initialization of yp variable when it is created

## Getting results for conf. D

### i) uncomment the constraint for 80% demand in scenario 1 specific constraints
### ii) comment the initialization of yp variable when it is created
### iii) uncomment the upgradation constraints
### iv) choose the 2nd objective function for minimization

### For conf. A to C, (iii) and (iv) are not applicable.

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from pulp import *
from math import *
#np.set_printoptions(precision=None,suppress=True)
#LpSolverDefault.msg = 1

# Read the Data from the Excel File

In [2]:
xls = pd.ExcelFile('/Users/ankit/Desktop/1316_ORCaseStudy (1)/1316_Network_Planning.xlsx')
plants = pd.read_excel(xls, 'Plants')
plant_product_pairs = pd.read_excel(xls, 'Plant Product Pairs')
plant_capacity = pd.read_excel(xls, 'Plant Capacity')
setups = pd.read_excel(xls, 'Setups')
customers = pd.read_excel(xls, 'Customers')
annual_demand = pd.read_excel(xls, 'Annual Demand')
customer_distances = pd.read_excel(xls, 'Customer Distances')
plant_distances = pd.read_excel(xls, 'Plant Distances')

In [3]:
# Useful Parameters

service_distance = 500
product_plant_assignment = {1:{1:1,2:0,3:0,4:0,5:0},  ## Outer keys are idedntifiers for plant
                            2:{1:0,2:1,3:0,4:0,5:0},  ## Inner keys are identifier for product
                            3:{1:0,2:0,3:1,4:0,5:0},  ## Inner Values: 0/1 - Whether the product is produced at the plant
                            4:{1:0,2:0,3:0,4:1,5:1}}

truck_capacity = 10  ## Tonnes
transp_cost_per_truck_per_mile = 2  ## $
setup_cost_per_day = 5000  ## $

production_rate = {
                    1:100,    ## Tonnes per hour
                    2:50,
                    3:50,
                    4:50
                }

plant_upgrade_cost = 10000000 ## $
w_hours = 8 ## Per day
overtime = 120  ## Hours per month
cost_factor_overtime = 50 ## Percent
n_months = 3
days_in_month = 30

bigM = 1000000
service_requirement = 0.8 #80 percent

# Define Indices

In [4]:
plants_index = list(plants["ID"]) # Set of Plants
customers_index = list(customers["ID"]) # Set of customers
products_index = list(plant_product_pairs["Product"])   # Set of Products
potential_WH_index = list(customers["ID"])  # Potential WH locations (all customer locations)

# Variables

### Quantity of product pr produced at plant p

In [5]:
q = LpVariable.dicts("quantity_p_pr",indexs=[(p,pr) for p in plants_index for pr in products_index],lowBound=0, cat = "Continuous")

### Indicator(0/1): Whether product pr is produced at plant p

#### Enforcement of production of products at certain plants only

#### If commented, the optimum product ---> optimim plant asignment for production will be identified.

#### If left uncommented, this constraint will ensure that product_plant_assignment is followed:
##### i) Product 1 --> Plant 1 only
##### ii) Product 2 --> Plant 2 only
##### iii) Product 3 --> Plant 3 only
##### iv) Products 4 and 5 --> Plant 4 only

In [6]:
yp = LpVariable.dicts("prodn_indicator_p_pr",indexs=[(p,pr) for p in plants_index for pr in products_index],cat = "Binary")

for p in plants_index:
    for pr in products_index:
        yp[(p,pr)] = product_plant_assignment[(p)][(pr)]

### Plant upgrade variable

In [7]:
upgrade = LpVariable.dicts("upgrade_indicator",indexs=[p for p in plants_index],cat='Binary')

### Indicator(0/1): Whether there is a production changeover at plant p from product pr1 to product pr2 

In [8]:
cp = LpVariable.dicts("changeover_indicator",indexs=[(p,pr1,pr2) for p in plants_index for pr1 in products_index for pr2 in products_index],cat="Binary")

### Quantity of product pr transported from plant p to customer c

In [9]:
x1 = LpVariable.dicts("quantity_p_pr_c",indexs=[(p,pr,c) for p in plants_index for pr in products_index for c in customers_index],lowBound=0, cat="Continuous")

### Quantity of product pr transported from plant p to warehouse w

In [10]:
x2 = LpVariable.dicts("quantity_p_pr_w",indexs=[(p,pr,w) for p in plants_index for pr in products_index for w in potential_WH_index],lowBound=0, cat="Continuous")

### Quantity of product pr transported from warehouse w to customer c

In [11]:
x3 = LpVariable.dicts("quantity_w_pr_c",indexs=[(w,pr,c) for w in potential_WH_index for pr in products_index for c in customers_index],lowBound=0, cat="Continuous")

### Indicator(0/1): Whether product pr is transferred from plant p to warehouse w

In [12]:
y2 = LpVariable.dicts("transfer_indicator_p_pr_w",indexs=[(p,pr,w) for p in plants_index for pr in products_index for w in potential_WH_index],cat="Binary")

# Known Parameters

### Quarterly demand of customer c for product p (tonnes)

In [13]:
annual_demand["Demand (in tonnes)"] = annual_demand["Demand (in tonnes)"]/4
demand_param = annual_demand[["Customer ID","Product ID","Demand (in tonnes)"]]
demand_param = demand_param.set_index(["Customer ID","Product ID"])['Demand (in tonnes)'].to_dict()

### Quarterly revenue from customer c for product p ($ - revenue/tonnes)

In [14]:
revenue_param = annual_demand[["Customer ID","Product ID","Revenue ($)"]]
revenue_param = revenue_param.set_index(["Customer ID","Product ID"])["Revenue ($)"].to_dict()

### Quarterly production capacity of plant p for product pr

In [15]:
capacity_param = plant_capacity[["Plant ID","Product ID","Annual Production Capacity"]]
capacity_param["Annual Production Capacity"] = capacity_param["Annual Production Capacity"]/4
capacity_param = capacity_param.set_index(["Plant ID","Product ID"])["Annual Production Capacity"].to_dict()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Cost of production of product pr at plant p ($ - cost/tonnes)

In [16]:
cost_param = plant_capacity[["Plant ID","Product ID","Production Cost"]]
cost_param = cost_param.set_index(["Plant ID","Product ID"])["Production Cost"].to_dict()

### Changeover time (in days) from product pr1 to product pr2

In [17]:
changeover_param = setups.set_index(["From Product ID","To Product ID"])["Duration (in days)"].to_dict()

### Distances (miles)

#### i) customer to customer distance
#### ii) warehouse to customer distance
#### iii) plant to customer distance

In [18]:
c_to_c_distances_param = customer_distances.set_index(["Customer ID","Customer ID.1"])["Distance"].to_dict()
w_to_c_distances_param = customer_distances.set_index(["Customer ID","Customer ID.1"])["Distance"].to_dict()
p_to_c_distances_param = plant_distances.set_index(["Plant ID","Customer ID"])["Distance"].to_dict()

# Costs

### Transportation costs
#### i) Plant to customer
#### ii) plant to warehouse
#### iii) warehouse to customer
#### iv) total transportation cost = (i) + (ii) (iii)

In [19]:
plant_to_customer_cost = 0.2*lpSum([x1[(p,pr,c)]*p_to_c_distances_param[p,c] for p in plants_index for pr in products_index for c in customers_index])
plant_to_warehouse_cost = 0.2*lpSum([x2[(p,pr,w)]*p_to_c_distances_param[p,w] for p in plants_index for pr in products_index for w in potential_WH_index])
warehouse_to_customer_cost = 0.2*lpSum([x3[(w,pr,c)]*w_to_c_distances_param[w,c] for w in potential_WH_index for pr in products_index for c in customers_index])
total_transportation_cost = lpSum([plant_to_customer_cost,plant_to_warehouse_cost,warehouse_to_customer_cost])

### Production Cost

In [20]:
production_cost = lpSum([q[(p,pr)]]*cost_param[(p,pr)] for p in plants_index for pr in products_index)

### Changeover cost

In [21]:
changeover_cost = 5000*lpSum(cp[(p,pr1,pr2)]*changeover_param[(pr1,pr2)] for p in plants_index for pr1 in products_index for pr2 in products_index)

### Revenue

In [22]:
revenue = lpSum(demand_param[c,pr]*revenue_param[c,pr] for pr in products_index for c in customers_index)

### Profit = Revenue - Costs

In [23]:
profit = revenue - (total_transportation_cost + production_cost + changeover_cost)

# Define a model object

In [24]:
model = LpProblem("OR_Problem",LpMinimize)

# Constraints

###  Flow balancing at plant nodes

In [25]:
for p in plants_index:
    for pr in products_index:
        model += lpSum([x1[(p,pr,c)] for c in customers_index]) + lpSum([x2[(p,pr,w)] for w in customers_index]) == q[(p,pr)]

###  Flow balancing at warehouse nodes

In [26]:
for w in customers_index:
    for pr in products_index:
        model += lpSum([x3[(w,pr,c)] for c in customers_index]) == lpSum([x2[(p,pr,w)] for p in plants_index])

### Flow balancing at customer nodes

In [27]:
for pr in products_index:
    for c in customers_index:
        model += lpSum([x1[(p,pr,c)] for p in plants_index]) + lpSum([x3[(w,pr,c)] for w in customers_index]) == demand_param[(c,pr)]

### Production time at each plant <= Available time

In [28]:
# write the rhs of constraint in terms of variables as well - abhi constant ke terms me hai
for p in plants_index:
    model += (lpSum([q[(p,pr)] for pr in products_index])/production_rate[(p)]) + 8*lpSum([cp[(p,pr1,pr2)]*changeover_param[(pr1,pr2)] for pr1 in products_index for pr2 in products_index]) <= n_months*(w_hours*days_in_month + overtime)

### Capacity constraint at plant nodes

In [29]:
for p in plants_index:
    for pr in products_index:
        model += q[(p,pr)] <= capacity_param[(p,pr)]

### Big M constraints for a decision variable and its indicator variable

#### i) Indicator(plant p ---> warehouse w; product pr) * M>= Quantity transferred(plant p ---> warehouse w; product pr)
#### ii) Indicator(plant p, product pr) * M >= Quantity produced(plant p, product pr)

In [30]:
for p in plants_index:
    for pr in products_index:
        for w in potential_WH_index:
            model += bigM*y2[(p,pr,w)] >= x2[(p,pr,w)]

In [31]:
for p in plants_index:
    for pr in products_index:
        model += bigM*yp[(p,pr)] >= q[(p,pr)]

### Constraints to ensure cyclicity of changeovers

In [32]:
for p in plants_index:
    for pr2 in products_index:
        model += lpSum([cp[(p,pr1,pr2)] for pr1 in products_index]) == yp[(p,pr2)]

In [33]:
for p in plants_index:
    for pr1 in products_index:
        model += lpSum(cp[(p,pr1,pr2)] for pr2 in products_index) == yp[(p,pr1)]

In [34]:
for p in plants_index:
    model += lpSum([cp[(p,pr,pr)] for pr in products_index]) <=1

In [35]:
for p in plants_index:
    model += lpSum([cp[(p,pr1,pr2)] for pr1 in products_index for pr2 in products_index]) == lpSum(yp[(p,pr)] for pr in products_index)

### Upgradation constraints

In [36]:
#for p in plants_index:
#    for pr in products_index:
#        model += yp[(p,pr)]<=upgrade[(p)]

# Scenario specific constraints

### Scenario 1: At least 80% of total demand is met by a facility(plant or warehouse) within 500 miles of the customer

In [37]:
# indicator for x1

p_to_c_distances_param_500 = {} # indicator for distance(plant, customer)<500

for p in plants_index:
    for c in customers_index:
        if p_to_c_distances_param[(p,c)]<500:
            p_to_c_distances_param_500[(p,c)] = 1
        else:
            p_to_c_distances_param_500[(p,c)] = 0

            
# indicator for x3

w_to_c_distances_param_500 = {} # indicator for distance(warehouse, customer)<500

for w in potential_WH_index:
    for c in customers_index:
        if w_to_c_distances_param[(w,c)]<500:
            w_to_c_distances_param_500[(w,c)] = 1
        else:
            w_to_c_distances_param_500[(w,c)] = 0

In [38]:
w_to_c = 0

for w in potential_WH_index:
    for c in customers_index:
        w_to_c = w_to_c + lpSum([x3[(w,pr,c)]*w_to_c_distances_param_500[(w,c)] for pr in products_index])

p_to_c = 0

for p in plants_index:
    for c in customers_index:
        p_to_c = p_to_c + lpSum([x1[(p,pr,c)]]*p_to_c_distances_param_500[p,c] for pr in products_index)

### Constraint Equation for scenario 1
#### Comment this cell to remove the 80% demand constraint


In [39]:
#model += w_to_c + p_to_c >= service_requirement*sum(demand_param.values())

# Objective Function

#### The objective is to increase the service level to customers that are far-off by building warehouses. This, however, leads to additional transportation and fixed costs and decreases the profit.

#### So, objective:

#### minimize (no. of warehouses - profit)

#### or

#### minimize(no. of warehouses - profit + no. of plants upgraded)

In [40]:
#w/o plant upgradation decision variable
model += 10*lpSum(y2) - profit/200000000

#with plant upgradation decision variable
#model += 10*lpSum(y2) - profit/200000000 + 10*lpSum(yp)

#model += 10*lpSum(y2) - profit/200000000 + 10*lpSum(upgrade)

# Solve the Problem

In [41]:
model.solve(PULP_CBC_CMD(mip=True, msg=True, timeLimit=500, fracGap=None, maxSeconds=None, gapRel=0.05, gapAbs=None, presolve=None, cuts=None, strong=None, options=None, warmStart=False, keepFiles=False, path=None, threads=None, logPath=None, mip_start=False))

1

# Results

#### Warehouse Locations

In [42]:
wh_loc = []

for p in plants_index:
    for pr in products_index:
        for w in potential_WH_index:
            if y2[(p,pr,w)].value()==1:
                wh_loc.append(w)

print("\nNo. of warehouses = ",len(np.unique(wh_loc)))
print("\nWarehouse Locations:",np.unique(wh_loc))


No. of warehouses =  0

Warehouse Locations: []


#### Product assignments

In [43]:
product_assignments = {p:{pr: value(yp[p,pr]) for pr in products_index} for p in plants_index}
product_assignments

{1: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0},
 2: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0},
 3: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0},
 4: {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}}

#### Production quantities

In [44]:
production_quantities = []

for p,pr in itertools.product(plants_index, products_index):
    production_quantities.append({"plant":p,"product":pr,"quantity":4*round(value(q[p,pr]),2)})

production_quantities

[{'plant': 1, 'product': 1, 'quantity': 295488.72},
 {'plant': 1, 'product': 2, 'quantity': 0.0},
 {'plant': 1, 'product': 3, 'quantity': 0.0},
 {'plant': 1, 'product': 4, 'quantity': 0.0},
 {'plant': 1, 'product': 5, 'quantity': 0.0},
 {'plant': 2, 'product': 1, 'quantity': 0.0},
 {'plant': 2, 'product': 2, 'quantity': 72580.16},
 {'plant': 2, 'product': 3, 'quantity': 0.0},
 {'plant': 2, 'product': 4, 'quantity': 0.0},
 {'plant': 2, 'product': 5, 'quantity': 0.0},
 {'plant': 3, 'product': 1, 'quantity': 0.0},
 {'plant': 3, 'product': 2, 'quantity': 0.0},
 {'plant': 3, 'product': 3, 'quantity': 29797.44},
 {'plant': 3, 'product': 4, 'quantity': 0.0},
 {'plant': 3, 'product': 5, 'quantity': 0.0},
 {'plant': 4, 'product': 1, 'quantity': 0.0},
 {'plant': 4, 'product': 2, 'quantity': 0.0},
 {'plant': 4, 'product': 3, 'quantity': 0.0},
 {'plant': 4, 'product': 4, 'quantity': 11922.08},
 {'plant': 4, 'product': 5, 'quantity': 5940.28}]

#### Costs, Revenue and Profits

In [45]:
costs = {'revenue': 4*round(value(revenue/1000000),2),
         'total_cost': 4*round(sum([value(total_transportation_cost),value(changeover_cost),value(production_cost)])/1000000,2),
         'profit'  :    4*round(value(profit/1000000),2)
                    }

costs

{'revenue': 431.2, 'total_cost': 288.76, 'profit': 142.44}

In [46]:
total_cost_breakup = {
    'total transportation_cost' : 4*round(value(total_transportation_cost/1000000),2),
    'changeover_cost'  : 4*round(value(changeover_cost/1000000),4),
    'production_cost'  : 4*round(value(production_cost/1000000),2)}

total_cost_breakup

{'total transportation_cost': 99.88,
 'changeover_cost': 0.18,
 'production_cost': 188.68}

In [47]:
transportation_cost_breakup = {
    'plant to customer shipping cost' : 4*round(value(plant_to_customer_cost/1000000),2),
    'plant to warehouse shipping cost'  : 4*round(value(plant_to_warehouse_cost/1000000),2),
    'warehouse to customer shipping cost'  : 4*round(value(warehouse_to_customer_cost/1000000),2)
}

transportation_cost_breakup

{'plant to customer shipping cost': 99.88,
 'plant to warehouse shipping cost': 0.0,
 'warehouse to customer shipping cost': 0.0}

#### Amounts delivered

In [48]:
first_mile = []
for p,pr,w in itertools.product(plants_index,products_index,potential_WH_index):
    first_mile.append({"from_plant":p, "to_warehouse":w, "product":pr, "quantity": 4*round(value(x2[p,pr,w]),2)})

last_mile_1 = []
for p,pr,c in itertools.product(plants_index, products_index, customers_index):
    last_mile_1.append({"from_plant":p, "to_customer":c, "product":pr,"quantity": 4*round(value(x1[p,pr,c]),2)})

last_mile_2 = []
for w,pr,c in itertools.product(potential_WH_index, products_index, customers_index):
    last_mile_2.append({"from_warehouse":w, "to_customer":c, "product":pr,"quantity":4*round(value(x3[w,pr,c]),2)})


In [49]:
#first_mile

In [50]:
#last_mile_1

In [51]:
#last_mile_2

#### No of trucks required

In [52]:
trucks_count_first_mile = 0

for i in range(len(first_mile)):
    trucks_count_first_mile = trucks_count_first_mile + np.ceil(first_mile[i]["quantity"]/10)
print("\nTrucks required from plants to warehouses:",trucks_count_first_mile)


trucks_count_last_mile_1 = 0
for i in range(len(last_mile_1)):
    trucks_count_last_mile_1 = trucks_count_last_mile_1 + np.ceil(last_mile_1[i]["quantity"]/10)
print("\nTrucks required from plants to customers:",trucks_count_last_mile_1)

trucks_count_last_mile_2 = 0
for i in range(len(last_mile_2)):
    trucks_count_last_mile_2 = trucks_count_last_mile_2 + np.ceil(last_mile_2[i]["quantity"]/10)

print("\nTrucks required from warehouses to customers:",trucks_count_last_mile_2)

print("\nTotal trucks required:",trucks_count_first_mile + trucks_count_last_mile_1 + trucks_count_last_mile_2)


Trucks required from plants to warehouses: 0.0

Trucks required from plants to customers: 41695.0

Trucks required from warehouses to customers: 0.0

Total trucks required: 41695.0


#### Service Percentage

In [53]:
print("Service Percentage: {} %".format(round(100*(value(w_to_c) + value(p_to_c))/sum(demand_param.values()),2)))

Service Percentage: 10.56 %


#### Changeovers (setups) required

In [54]:
changeover_indicator = []
for p,pr1,pr2 in itertools.product(plants_index,products_index,products_index):
    if value(cp[p,pr1,pr2]) == 1:
        changeover_indicator.append({"plant":p, "from product":pr1, "to product":pr2, "setup required": value(cp[p,pr1,pr2])})

changeover_indicator

[{'plant': 1, 'from product': 1, 'to product': 1, 'setup required': 1.0},
 {'plant': 2, 'from product': 2, 'to product': 2, 'setup required': 1.0},
 {'plant': 3, 'from product': 3, 'to product': 3, 'setup required': 1.0},
 {'plant': 4, 'from product': 4, 'to product': 5, 'setup required': 1.0},
 {'plant': 4, 'from product': 5, 'to product': 4, 'setup required': 1.0}]

### Plants to be upgraded

In [55]:
for p in plants_index:
    print("Upgrade plant",p, value(upgrade[(p)]))

Upgrade plant 1 None
Upgrade plant 2 None
Upgrade plant 3 None
Upgrade plant 4 None


In [56]:
#writer = pd.ExcelWriter('/Users/ankit/Desktop/OR Case Study Solution/Only_Plant_upgrades.xlsx', engine='xlsxwriter')
#df_costs = pd.DataFrame(costs,index=['0'])
#df_costs.to_excel(writer,sheet_name = 'Balance Sheet',index = False)
#df_total_cost_breakup = pd.DataFrame(total_cost_breakup,index=['0'])
#df_total_cost_breakup.to_excel(writer, sheet_name='Total Costs Breakup',index=False)
#df_transportation_cost_breakup = pd.DataFrame(transportation_cost_breakup,index=['0'])
#df_transportation_cost_breakup.to_excel(writer, sheet_name='Transportation Costs Breakup',index=False)
#df_product_assignments = pd.DataFrame(product_assignments)
#df_product_assignments.to_excel(writer, sheet_name='Product Assignments',index=False)
#df_production_quantities = pd.DataFrame(production_quantities)
#df_production_quantities.to_excel(writer, sheet_name='Production Quantities',index=False)
#df_first_mile = pd.DataFrame(first_mile)
#df_first_mile.to_excel(writer, sheet_name='Qty - Plants to Warehouse',index=False)
#df_last_mile_1 = pd.DataFrame(last_mile_1)
#df_last_mile_1.to_excel(writer, sheet_name='Qty - Plants to Customer',index=False)
#df_last_mile_2 = pd.DataFrame(last_mile_2)
#df_last_mile_2.to_excel(writer, sheet_name='Qty - Warehouse to Customer',index=False)
#writer.save()